In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import fastparquet
import pyarrow

In [2]:
orders = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/orders.csv')
order_items = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/order_items.csv')
products = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/products.csv')
website_sessions = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/website_sessions.csv')
order_item_refunds = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/order_item_refunds.csv')
website_pageviews = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/website_pageviews.csv')
maven_fuzzy_factory_data_dictionary = pd.read_csv('~/Downloads/Maven+Fuzzy+Factory/maven_fuzzy_factory_data_dictionary.csv')

In [24]:
maven_fuzzy_factory_data_dictionary

,Table,Field,Description
0,orders,order_id,Unique identifier for each order (PK)
1,orders,created_at,Timestamp when the order was placed
2,orders,website_session_id,Unique identifier for the website session (FK)
3,orders,user_id,Unique identifier for the user (FK)
4,orders,primary_product_id,Unique identifier for the primary product in t...
5,orders,items_purchased,Number of items in the order
6,orders,price_usd,Total price for the items in the order
7,orders,cogs_usd,Cost of goods sold for the items in the order
8,order_items,order_item_id,Unique identifier for each order item (PK)
9,order_items,created_at,Timestamp when the order was placed


In [25]:
# ==================== DATA LOADING ====================
print("Loading datasets...")
print(f"Orders: {orders.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Products: {products.shape}")
print(f"Website Sessions: {website_sessions.shape}")
print(f"Order Item Refunds: {order_item_refunds.shape}")
print(f"Website Pageviews: {website_pageviews.shape}")

Loading datasets...
Orders: (32313, 8)
Order Items: (40025, 7)
Products: (4, 3)
Website Sessions: (472871, 9)
Order Item Refunds: (1731, 5)
Website Pageviews: (1188124, 4)


In [26]:
# ==================== DATA EXPLORATION ====================
print("\n=== ORDERS DATASET ===")
print(f"Shape: {orders.shape}")
print(f"Duplicates: {orders.duplicated().sum()}")
print(f"Missing values:\n{orders.isnull().sum()}")
print(f"\nData types:\n{orders.dtypes}")

print("\n=== ORDER ITEMS DATASET ===")
print(f"Shape: {order_items.shape}")
print(f"Duplicates: {order_items.duplicated().sum()}")
print(f"Missing values:\n{order_items.isnull().sum()}")

print("\n=== PRODUCTS DATASET ===")
print(f"Shape: {products.shape}")
print(f"Duplicates: {products.duplicated().sum()}")
print(f"Missing values:\n{products.isnull().sum()}")
print(f"\nProducts:\n{products}")

print("\n=== WEBSITE SESSIONS DATASET ===")
print(f"Shape: {website_sessions.shape}")
print(f"Duplicates: {website_sessions.duplicated().sum()}")
print(f"Missing values:\n{website_sessions.isnull().sum()}")

print("\n=== ORDER ITEM REFUNDS DATASET ===")
print(f"Shape: {order_item_refunds.shape}")
print(f"Duplicates: {order_item_refunds.duplicated().sum()}")
print(f"Missing values:\n{order_item_refunds.isnull().sum()}")

print("\n=== WEBSITE PAGEVIEWS DATASET ===")
print(f"Shape: {website_pageviews.shape}")
print(f"Duplicates: {website_pageviews.duplicated().sum()}")
print(f"Missing values:\n{website_pageviews.isnull().sum()}")


=== ORDERS DATASET ===
Shape: (32313, 8)
Duplicates: 0
Missing values:
order_id              0
created_at            0
website_session_id    0
user_id               0
primary_product_id    0
items_purchased       0
price_usd             0
cogs_usd              0
dtype: int64

Data types:
order_id                int64
created_at             object
website_session_id      int64
user_id                 int64
primary_product_id      int64
items_purchased         int64
price_usd             float64
cogs_usd              float64
dtype: object

=== ORDER ITEMS DATASET ===
Shape: (40025, 7)
Duplicates: 0
Missing values:
order_item_id      0
created_at         0
order_id           0
product_id         0
is_primary_item    0
price_usd          0
cogs_usd           0
dtype: int64

=== PRODUCTS DATASET ===
Shape: (4, 3)
Duplicates: 0
Missing values:
product_id      0
created_at      0
product_name    0
dtype: int64

Products:
   product_id           created_at                product_name
0       

In [27]:
# ==================== DATA CLEANING & DEDUPLICATION ====================

# Convert datetime columns
orders_clean = orders.copy()
orders_clean['created_at'] = pd.to_datetime(orders_clean['created_at'])

order_items_clean = order_items.copy()
order_items_clean['created_at'] = pd.to_datetime(order_items_clean['created_at'])

products_clean = products.copy()
products_clean['created_at'] = pd.to_datetime(products_clean['created_at'])

website_sessions_clean = website_sessions.copy()
website_sessions_clean['created_at'] = pd.to_datetime(website_sessions_clean['created_at'])

order_item_refunds_clean = order_item_refunds.copy()
order_item_refunds_clean['created_at'] = pd.to_datetime(order_item_refunds_clean['created_at'])

website_pageviews_clean = website_pageviews.copy()
website_pageviews_clean['created_at'] = pd.to_datetime(website_pageviews_clean['created_at'])

# Remove duplicates based on primary keys
orders_clean = orders_clean.drop_duplicates(subset=['order_id'], keep='first')
order_items_clean = order_items_clean.drop_duplicates(subset=['order_item_id'], keep='first')
products_clean = products_clean.drop_duplicates(subset=['product_id'], keep='first')
website_sessions_clean = website_sessions_clean.drop_duplicates(subset=['website_session_id'], keep='first')
order_item_refunds_clean = order_item_refunds_clean.drop_duplicates(subset=['order_item_refund_id'], keep='first')
website_pageviews_clean = website_pageviews_clean.drop_duplicates(subset=['website_pageview_id'], keep='first')

# Handle missing values in website_sessions (UTM parameters can be NULL)
website_sessions_clean['utm_source'] = website_sessions_clean['utm_source'].fillna('direct')
website_sessions_clean['utm_campaign'] = website_sessions_clean['utm_campaign'].fillna('none')
website_sessions_clean['utm_content'] = website_sessions_clean['utm_content'].fillna('none')
website_sessions_clean['http_referer'] = website_sessions_clean['http_referer'].fillna('none')

# Add date/time features for analysis
orders_clean['order_date'] = orders_clean['created_at'].dt.date
orders_clean['order_month'] = orders_clean['created_at'].dt.to_period('M')
orders_clean['order_year'] = orders_clean['created_at'].dt.year
orders_clean['order_quarter'] = orders_clean['created_at'].dt.quarter
orders_clean['order_hour'] = orders_clean['created_at'].dt.hour
orders_clean['order_dayofweek'] = orders_clean['created_at'].dt.dayofweek

# Calculate profit
orders_clean['profit_usd'] = orders_clean['price_usd'] - orders_clean['cogs_usd']
order_items_clean['profit_usd'] = order_items_clean['price_usd'] - order_items_clean['cogs_usd']

print("Data cleaning completed!")
print(f"\nOrders after cleaning: {orders_clean.shape}")
print(f"Order Items after cleaning: {order_items_clean.shape}")
print(f"Products after cleaning: {products_clean.shape}")
print(f"Website Sessions after cleaning: {website_sessions_clean.shape}")
print(f"Order Item Refunds after cleaning: {order_item_refunds_clean.shape}")
print(f"Website Pageviews after cleaning: {website_pageviews_clean.shape}")

Data cleaning completed!

Orders after cleaning: (32313, 15)
Order Items after cleaning: (40025, 8)
Products after cleaning: (4, 3)
Website Sessions after cleaning: (472871, 9)
Order Item Refunds after cleaning: (1731, 5)
Website Pageviews after cleaning: (1188124, 4)


In [28]:
# ==================== JOINING TABLES ====================

# Main analysis table: Orders with session data
orders_with_sessions = orders_clean.merge(
    website_sessions_clean,
    on='website_session_id',
    how='left',
    suffixes=('_order', '_session')
)

# Add product information
orders_with_sessions = orders_with_sessions.merge(
    products_clean,
    left_on='primary_product_id',
    right_on='product_id',
    how='left',
    suffixes=('', '_product')
)

# Create order items with product details
order_items_full = order_items_clean.merge(
    products_clean,
    on='product_id',
    how='left',
    suffixes=('', '_product')
)

# Add order information to order items
order_items_full = order_items_full.merge(
    orders_clean[['order_id', 'website_session_id', 'user_id', 'order_date', 'order_month', 'order_year']],
    on='order_id',
    how='left'
)

# Add refund information to order items
order_items_full = order_items_full.merge(
    order_item_refunds_clean[['order_item_id', 'refund_amount_usd']],
    on='order_item_id',
    how='left'
)
order_items_full['is_refunded'] = order_items_full['refund_amount_usd'].notna().astype(int)

# Create session-level pageview counts
pageview_counts = website_pageviews_clean.groupby('website_session_id').agg({
    'website_pageview_id': 'count'
}).rename(columns={'website_pageview_id': 'pageviews_count'}).reset_index()

# Add pageview counts to orders
orders_with_sessions = orders_with_sessions.merge(
    pageview_counts,
    on='website_session_id',
    how='left'
)
orders_with_sessions['pageviews_count'] = orders_with_sessions['pageviews_count'].fillna(0)

print("Table joining completed!")
print(f"\nOrders with Sessions: {orders_with_sessions.shape}")
print(f"Order Items Full: {order_items_full.shape}")
print(f"\nSample of joined data:")
print(orders_with_sessions.head())

Table joining completed!

Orders with Sessions: (32313, 27)
Order Items Full: (40025, 17)

Sample of joined data:
   order_id    created_at_order  website_session_id  user_id_order  \
0         1 2012-03-19 10:42:46                  20             20   
1         2 2012-03-19 19:27:37                 104            104   
2         3 2012-03-20 06:44:45                 147            147   
3         4 2012-03-20 09:41:45                 160            160   
4         5 2012-03-20 11:28:15                 177            177   

   primary_product_id  items_purchased  price_usd  cogs_usd  order_date  \
0                   1                1      49.99     19.49  2012-03-19   
1                   1                1      49.99     19.49  2012-03-19   
2                   1                1      49.99     19.49  2012-03-20   
3                   1                1      49.99     19.49  2012-03-20   
4                   1                1      49.99     19.49  2012-03-20   

  order_month 

In [8]:
# ==================== PLOTLY VISUALIZATIONS ====================

# 1. Revenue Over Time
revenue_by_month = orders_with_sessions.groupby(orders_with_sessions['created_at_order'].dt.to_period('M')).agg({
    'price_usd': 'sum',
    'profit_usd': 'sum',
    'order_id': 'count'
}).reset_index()
revenue_by_month['created_at_order'] = revenue_by_month['created_at_order'].astype(str)

fig1 = px.line(revenue_by_month, 
               x='created_at_order', 
               y=['price_usd', 'profit_usd'],
               title='Revenue and Profit Trends Over Time',
               labels={'value': 'Amount (USD)', 'created_at_order': 'Month', 'variable': 'Metric'},
               template='plotly_white')
fig1.update_layout(height=500, showlegend=True, hovermode='x unified')
fig1.show()

# 2. Orders Over Time
fig2 = px.bar(revenue_by_month,
              x='created_at_order',
              y='order_id',
              title='Number of Orders Over Time',
              labels={'order_id': 'Number of Orders', 'created_at_order': 'Month'},
              template='plotly_white')
fig2.update_layout(height=500)
fig2.show()

In [29]:
# 3. Product Performance
product_performance = order_items_full.groupby('product_name').agg({
    'order_item_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum',
    'is_refunded': 'sum'
}).reset_index()
product_performance.columns = ['product_name', 'units_sold', 'revenue', 'profit', 'refunds']
product_performance['refund_rate'] = (product_performance['refunds'] / product_performance['units_sold'] * 100).round(2)

fig3 = px.bar(product_performance.sort_values('revenue', ascending=False),
              x='product_name',
              y='revenue',
              title='Revenue by Product',
              labels={'revenue': 'Revenue (USD)', 'product_name': 'Product'},
              text='revenue',
              template='plotly_white',
              color='revenue',
              color_continuous_scale='Blues')
fig3.update_traces(texttemplate='$%{text:.2f}', textposition='outside')
fig3.update_layout(height=500)
fig3.show()

# 4. Product Units Sold
fig4 = px.pie(product_performance,
              values='units_sold',
              names='product_name',
              title='Market Share by Units Sold',
              template='plotly_white',
              hole=0.4)
fig4.update_traces(textposition='inside', textinfo='percent+label')
fig4.update_layout(height=1000)
fig4.show()

In [30]:
product_performance

,product_name,units_sold,revenue,profit,refunds,refund_rate
0,The Birthday Sugar Panda,4985,229260.15,157027.5,301,6.04
1,The Forever Love Bear,5796,347702.04,217350.0,129,2.23
2,The Hudson River Mini bear,5018,150489.82,102869.0,64,1.28
3,The Original Mr. Fuzzy,24226,1211057.74,738893.0,1237,5.11


In [10]:
# 5. Traffic Source Analysis
traffic_analysis = orders_with_sessions.groupby('utm_source').agg({
    'order_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum'
}).reset_index()

traffic_analysis.columns = ['utm_source', 'orders', 'revenue', 'profit']
traffic_analysis = traffic_analysis.sort_values('revenue', ascending=False)

fig5 = px.bar(traffic_analysis,
              x='utm_source',
              y=['orders', 'revenue'],
              title='Orders and Revenue by Traffic Source',
              labels={'value': 'Count / Amount', 'utm_source': 'Traffic Source', 'variable': 'Metric'},
              barmode='group',
              template='plotly_white')
fig5.update_layout(height=500)
fig5.show()

# 6. Campaign Performance
campaign_analysis = orders_with_sessions.groupby('utm_campaign').agg({
    'order_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum'
}).reset_index()
campaign_analysis.columns = ['utm_campaign', 'orders', 'revenue', 'profit']
campaign_analysis = campaign_analysis.sort_values('revenue', ascending=False).head(10)

fig6 = px.scatter(campaign_analysis,
                  x='orders',
                  y='profit',
                  size='revenue',
                  color='utm_campaign',
                  title='Top 10 Campaigns: Orders vs Profit (bubble size = revenue)',
                  labels={'orders': 'Number of Orders', 'profit': 'Profit (USD)'},
                  template='plotly_white',
                  hover_data=['revenue'])
fig6.update_layout(height=500, showlegend=True)
fig6.show()

In [31]:
# 7. Device Type Analysis
device_analysis = orders_with_sessions.groupby('device_type').agg({
    'order_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum'
}).reset_index()
device_analysis.columns = ['device_type', 'orders', 'revenue', 'profit']

fig7 = px.bar(device_analysis,
              x='device_type',
              y=['orders', 'revenue', 'profit'],
              title='Performance by Device Type',
              labels={'value': 'Count / Amount (USD)', 'device_type': 'Device Type', 'variable': 'Metric'},
              barmode='group',
              template='plotly_white')
fig7.update_layout(height=500)
fig7.show()

# 8. Repeat Customer Analysis
repeat_customer = orders_with_sessions.groupby('is_repeat_session').agg({
    'order_id': 'count',
    'price_usd': 'sum'
}).reset_index()
repeat_customer['is_repeat_session'] = repeat_customer['is_repeat_session'].map({0: 'New Customer', 1: 'Repeat Customer'})
repeat_customer.columns = ['customer_type', 'orders', 'revenue']

fig8 = px.pie(repeat_customer,
              values='revenue',
              names='customer_type',
              title='Revenue Split: New vs Repeat Customers',
              template='plotly_white',
              hole=0.4,
              color_discrete_sequence=px.colors.qualitative.Set2)
fig8.update_traces(textposition='inside', textinfo='percent+label+value', texttemplate='%{label}<br>%{percent}<br>$%{value:,.0f}')
fig8.update_layout(height=1100)
fig8.show()

In [12]:
# 9. Order Time Analysis (Hour of Day)
hourly_orders = orders_with_sessions.groupby('order_hour').agg({
    'order_id': 'count',
    'price_usd': 'sum'
}).reset_index()
hourly_orders.columns = ['hour', 'orders', 'revenue']

fig9 = px.line(hourly_orders,
               x='hour',
               y='orders',
               title='Order Volume by Hour of Day',
               labels={'orders': 'Number of Orders', 'hour': 'Hour of Day'},
               template='plotly_white',
               markers=True)
fig9.update_layout(height=500)
fig9.show()

# 10. Day of Week Analysis
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
dow_orders = orders_with_sessions.groupby('order_dayofweek').agg({
    'order_id': 'count',
    'price_usd': 'sum'
}).reset_index()
dow_orders['day_name'] = dow_orders['order_dayofweek'].map(day_names)
dow_orders.columns = ['day_num', 'orders', 'revenue', 'day_name']

fig10 = px.bar(dow_orders,
               x='day_name',
               y='orders',
               title='Order Volume by Day of Week',
               labels={'orders': 'Number of Orders', 'day_name': 'Day of Week'},
               template='plotly_white',
               color='orders',
               color_continuous_scale='Viridis')
fig10.update_layout(height=500)
fig10.show()

In [13]:
# 11. Refund Analysis
refund_summary = order_items_full.groupby('product_name').agg({
    'order_item_id': 'count',
    'is_refunded': 'sum',
    'refund_amount_usd': 'sum'
}).reset_index()
refund_summary.columns = ['product_name', 'total_items', 'refunded_items', 'refund_amount']
refund_summary['refund_rate'] = (refund_summary['refunded_items'] / refund_summary['total_items'] * 100).round(2)

fig11 = px.bar(refund_summary,
               x='product_name',
               y='refund_rate',
               title='Refund Rate by Product (%)',
               labels={'refund_rate': 'Refund Rate (%)', 'product_name': 'Product'},
               template='plotly_white',
               color='refund_rate',
               color_continuous_scale='Reds',
               text='refund_rate')
fig11.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig11.update_layout(height=500)
fig11.show()

# 12. Refund Amount Over Time
refunds_by_month = order_items_full[order_items_full['is_refunded'] == 1].groupby(
    order_items_full[order_items_full['is_refunded'] == 1]['order_month']
).agg({
    'refund_amount_usd': 'sum',
    'order_item_id': 'count'
}).reset_index()
refunds_by_month['order_month'] = refunds_by_month['order_month'].astype(str)
refunds_by_month.columns = ['month', 'refund_amount', 'refund_count']

fig12 = px.line(refunds_by_month,
                x='month',
                y=['refund_amount', 'refund_count'],
                title='Refunds Over Time',
                labels={'value': 'Amount / Count', 'month': 'Month', 'variable': 'Metric'},
                template='plotly_white',
                markers=True)
fig12.update_layout(height=500)
fig12.show()

In [14]:
# 13. Conversion Analysis - Pageviews vs Orders
conversion_data = orders_with_sessions[['pageviews_count', 'price_usd', 'product_name']].copy()
conversion_data = conversion_data[conversion_data['pageviews_count'] > 0]

fig13 = px.scatter(conversion_data,
                   x='pageviews_count',
                   y='price_usd',
                   color='product_name',
                   title='Order Value vs Number of Pageviews',
                   labels={'pageviews_count': 'Number of Pageviews', 'price_usd': 'Order Value (USD)', 'product_name': 'Product'},
                   template='plotly_white',
                   opacity=0.6)
fig13.update_layout(height=500)
fig13.show()

# 14. Monthly Growth Rate
monthly_metrics = orders_with_sessions.groupby(orders_with_sessions['created_at_order'].dt.to_period('M')).agg({
    'order_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum'
}).reset_index()
monthly_metrics['created_at_order'] = monthly_metrics['created_at_order'].astype(str)
monthly_metrics.columns = ['month', 'orders', 'revenue', 'profit']

# Calculate growth rates
monthly_metrics['revenue_growth'] = monthly_metrics['revenue'].pct_change() * 100
monthly_metrics['order_growth'] = monthly_metrics['orders'].pct_change() * 100

fig14 = px.line(monthly_metrics.iloc[1:],  # Skip first month (no growth rate)
                x='month',
                y=['revenue_growth', 'order_growth'],
                title='Month-over-Month Growth Rates (%)',
                labels={'value': 'Growth Rate (%)', 'month': 'Month', 'variable': 'Metric'},
                template='plotly_white',
                markers=True)
fig14.update_layout(height=500, hovermode='x unified')
fig14.show()

In [15]:
# 15. Customer Lifetime Value (CLV) Analysis
# Handle user_id column - it may have a suffix from merge
user_id_col = 'user_id_order' if 'user_id_order' in orders_with_sessions.columns else 'user_id'

customer_clv = orders_with_sessions.groupby(user_id_col).agg({
    'order_id': 'count',
    'price_usd': 'sum',
    'profit_usd': 'sum'
}).reset_index()
customer_clv.columns = ['user_id', 'order_count', 'total_revenue', 'total_profit']
customer_clv['avg_order_value'] = (customer_clv['total_revenue'] / customer_clv['order_count']).round(2)

# Distribution of order counts
order_count_dist = customer_clv['order_count'].value_counts().sort_index().reset_index()
order_count_dist.columns = ['orders_per_customer', 'customer_count']

fig15 = px.bar(order_count_dist.head(20),
               x='orders_per_customer',
               y='customer_count',
               title='Customer Order Frequency Distribution (Top 20)',
               labels={'orders_per_customer': 'Number of Orders', 'customer_count': 'Number of Customers'},
               template='plotly_white')
fig15.update_layout(height=500)
fig15.show()

# CLV distribution
fig16 = px.histogram(customer_clv,
                     x='total_revenue',
                     nbins=50,
                     title='Customer Lifetime Value Distribution',
                     labels={'total_revenue': 'Total Revenue (USD)', 'count': 'Number of Customers'},
                     template='plotly_white')
fig16.update_layout(height=500)
fig16.show()

In [16]:
# ==================== PRE-PROCESS FOR PARQUET EXPORT ====================
# Convert Period columns NOW to avoid parquet issues later

# Fix orders_clean
if 'order_month' in orders_clean.columns and str(orders_clean['order_month'].dtype).startswith('period'):
    orders_clean['order_month'] = orders_clean['order_month'].astype(str)

# Fix order_items_full  
if 'order_month' in order_items_full.columns and str(order_items_full['order_month'].dtype).startswith('period'):
    order_items_full['order_month'] = order_items_full['order_month'].astype(str)

# Fix orders_with_sessions
if 'order_month' in orders_with_sessions.columns and str(orders_with_sessions['order_month'].dtype).startswith('period'):
    orders_with_sessions['order_month'] = orders_with_sessions['order_month'].astype(str)

print("✓ Converted Period columns to strings for parquet compatibility")

✓ Converted Period columns to strings for parquet compatibility


In [32]:
# ==================== SAVE TO PARQUET FORMAT (CSV BACKUP) ====================

import os

# Create output directory if it doesn't exist
output_dir = os.path.expanduser('~/Downloads/Maven+Fuzzy+Factory/cleaned_data')
os.makedirs(output_dir, exist_ok=True)

print("Saving cleaned datasets to Parquet/CSV format...")

# Helper to safely save with fallback to CSV
def safe_save(df, name, output_dir):
    try:
        # Try parquet first
        df.to_parquet(f'{output_dir}/{name}.parquet', index=False)
        print(f"✓ Saved: {name}.parquet ({df.shape})")
        return True
    except Exception as e:
        # Fallback to CSV if parquet fails
        print(f"⚠ Parquet failed for {name}, saving as CSV: {str(e)[:50]}")
        df.to_csv(f'{output_dir}/{name}.csv', index=False)
        print(f"✓ Saved: {name}.csv ({df.shape})")
        return False

# Save cleaned individual tables
safe_save(orders_clean, 'orders_clean', output_dir)
safe_save(order_items_clean, 'order_items_clean', output_dir)
safe_save(products_clean, 'products_clean', output_dir)
safe_save(website_sessions_clean, 'website_sessions_clean', output_dir)
safe_save(order_item_refunds_clean, 'order_item_refunds_clean', output_dir)
safe_save(website_pageviews_clean, 'website_pageviews_clean', output_dir)

# Save joined tables
safe_save(orders_with_sessions, 'orders_with_sessions', output_dir)
safe_save(order_items_full, 'order_items_full', output_dir)

# Save analysis datasets
safe_save(product_performance, 'product_performance', output_dir)
safe_save(customer_clv, 'customer_clv', output_dir)

print(f"\n✅ All datasets saved successfully to: {output_dir}")
print(f"\nTotal files saved: 10")
print("\nFile sizes:")
for file in sorted(os.listdir(output_dir)):
    if file.endswith('.parquet') or file.endswith('.csv'):
        size = os.path.getsize(f'{output_dir}/{file}') / 1024 / 1024  # Convert to MB
        print(f"  {file}: {size:.2f} MB")

Saving cleaned datasets to Parquet/CSV format...
✓ Saved: orders_clean.parquet ((32313, 15))
✓ Saved: order_items_clean.parquet ((40025, 8))
✓ Saved: products_clean.parquet ((4, 3))
✓ Saved: website_sessions_clean.parquet ((472871, 9))
✓ Saved: order_item_refunds_clean.parquet ((1731, 5))
✓ Saved: website_pageviews_clean.parquet ((1188124, 4))
✓ Saved: orders_with_sessions.parquet ((32313, 27))
✓ Saved: order_items_full.parquet ((40025, 17))
✓ Saved: product_performance.parquet ((4, 6))
✓ Saved: customer_clv.parquet ((31696, 5))

✅ All datasets saved successfully to: /Users/denistkachenko/Downloads/Maven+Fuzzy+Factory/cleaned_data

Total files saved: 10

File sizes:
  customer_clv.parquet: 0.24 MB
  order_item_refunds_clean.parquet: 0.05 MB
  order_items_clean.parquet: 0.79 MB
  order_items_full.parquet: 1.22 MB
  orders_clean.parquet: 0.94 MB
  orders_with_sessions.parquet: 1.50 MB
  product_performance.parquet: 0.00 MB
  products_clean.parquet: 0.00 MB
  website_pageviews_clean.parqu